# Probe Data - Map Matching
## Nick Paras | Kapil Garg

### Assignment 2

Input: Probe data and map [probe_data_map_matching.rar](https://canvas.northwestern.edu/courses/51440/files/3334329/download?wrap=1)

-The raw probe points in Germany collected in 9 months

-The link data for the links that probe points can be map-matched to.

Tasks:
-- map match probe points to road links

-- derive road slope for each road link

-- evaluate the derived road slope with the surveyed road slope in the link data file

**Please submit your code and slides presentation of your approach and results including evaluation comparing with the slopes in the link data file**

### Setup

We use **Python 3.6** and rely on the dependencies:
* numpy
* scikit-learn
* matplotlib
* pandas

We also use Jupyter Notebooks for our code and reports. For quick setup, please create a conda environment with the following:

    $ conda create --name probe-data pandas matplotlib numpy scikit-learn

and then activate the conda environment with

    $ source activate probe-data


In [221]:
# Imports
import os
import math
import csv
import operator
import multiprocessing as mp
import itertools
import time
import json
import gmplot

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from datetime import datetime
from haversine import haversine
from functools import reduce
from IPython.display import IFrame
from collections import namedtuple

# Custom classes
import link_classes as lc
import graph_classes as graph

from dist_functions import dist_to_link as link_dist


%matplotlib inline

# Constants
DATA_DIR = '../data'
GOOGLE_MAPS_KEY = ''
with open('config.json') as data_file:
    data = json.load(data_file)
    GOOGLE_MAPS_KEY = data['google-maps-key']

In [222]:
# Utility functions
def bearing(start, end):
    """
    Computes the bearing in degrees between two geopoints
    
    Inputs:
        start (tuple of lat, long): starting geolocation
        end (tuple of lat, long): ending geolocation
    
    Outputs:
        (float): bearing in degrees between start and end
    """
    phi_1 = math.radians(start[0])
    phi_2 = math.radians(end[0])
    lambda_1 = math.radians(start[1])
    lambda_2 = math.radians(end[1])
    
    x = math.cos(phi_2) * math.sin(lambda_2 - lambda_1)
    y = math.cos(phi_1) * math.sin(phi_2) - (math.sin(phi_1) * math.cos(phi_2) * math.cos(lambda_2 - lambda_1))

    return (math.degrees(math.atan2(x, y)) + 360) % 360

## Loading Probe Data for Map Matching

Here we'll load our data from the two csv's into Pandas DataFrames.

In [223]:
probe_headers = ['sampleID', 
                 'dateTime', 
                 'sourceCode', 
                 'latitude', 
                 'longitude', 
                 'altitude', 
                 'speed', 
                 'heading']

probe_data = pd.read_csv(os.path.join(DATA_DIR, 'Partition6467ProbePoints.csv'), header=None, names=probe_headers)
probe_data.drop_duplicates(inplace=True)
probe_data['id'] = probe_data['sampleID'].map(str) + '_' + probe_data['dateTime']
probe_data['dateTime'] = pd.to_datetime(probe_data['dateTime'], format='%m/%d/%Y %I:%M:%S %p')
probe_data.sort_values(['sampleID', 'dateTime'], ascending=[True, True], inplace=True)
probe_data.head()

,sampleID,dateTime,sourceCode,latitude,longitude,altitude,speed,heading,id
0,3496,2009-06-12 06:12:49,13,51.496868,9.386022,200,23,339,3496_6/12/2009 6:12:49 AM
1,3496,2009-06-12 06:12:54,13,51.496682,9.386157,200,10,129,3496_6/12/2009 6:12:54 AM
2,3496,2009-06-12 06:12:59,13,51.496705,9.386422,201,21,60,3496_6/12/2009 6:12:59 AM
3,3496,2009-06-12 06:13:04,13,51.496749,9.386840,201,0,360,3496_6/12/2009 6:13:04 AM
4,3496,2009-06-12 06:13:09,13,51.496864,9.387294,199,0,360,3496_6/12/2009 6:13:09 AM


In [224]:
link_headers = ['linkPVID', 
                'refNodeID', 
                'nrefNodeID', 
                'length', 
                'functionalClass', 
                'directionOfTravel', 
                'speedCategory', 
                'fromRefSpeedLimit', 
                'toRefSpeedLimit', 
                'fromRefNumLanes', 
                'toRefNumLanes', 
                'multiDigitized', 
                'urban', 
                'timeZone', 
                'shapeInfo', 
                'curvatureInfo', 
                'slopeInfo']

# load raw link data
link_data = pd.read_csv(os.path.join(DATA_DIR, 'Partition6467LinkData.csv'), header=None, names=link_headers)
link_data['speedLimit'] = link_data[['fromRefSpeedLimit', 'toRefSpeedLimit']].max(axis=1)
link_data.loc[link_data['speedLimit'] == 0, 'speedLimit'] = 1
link_data['shapeArray'] = link_data['shapeInfo'].apply(lambda x: [[float(j) for j in i.split('/')[:2]] for i in x.split('|')])
link_data['location'] = link_data['shapeArray'].apply(lambda x: reduce(np.add, x) / len(x))
link_data.head()

,linkPVID,refNodeID,nrefNodeID,length,functionalClass,directionOfTravel,speedCategory,fromRefSpeedLimit,toRefSpeedLimit,fromRefNumLanes,toRefNumLanes,multiDigitized,urban,timeZone,shapeInfo,curvatureInfo,slopeInfo,speedLimit,shapeArray,location
0,62007637,162844982,162809070,335.04,5,B,7,30,30,0,0,F,T,0.0,51.4965800/9.3862299/|51.4994700/9.3848799/,NaN,NaN,30,"[[51.49658, 9.3862299], [51.49947, 9.3848799]]","[51.498025, 9.3855549]"
1,567329767,162844982,162981512,134.56,5,B,7,0,0,0,0,F,T,0.0,51.4965800/9.3862299/|51.4966899/9.3867100/|51...,NaN,NaN,1,"[[51.49658, 9.3862299], [51.4966899, 9.38671],...","[51.496769975, 9.387074925]"
2,62007648,162877732,162844982,97.01,5,B,7,30,30,0,0,F,T,0.0,51.4962899/9.3849100/|51.4965800/9.3862299/,NaN,NaN,30,"[[51.4962899, 9.38491], [51.49658, 9.3862299]]","[51.49643495, 9.38556995]"
3,78670326,162877732,163152693,314.84,5,B,7,30,30,0,0,F,T,0.0,51.4962899/9.3849100/|51.4990000/9.3836099/,NaN,NaN,30,"[[51.4962899, 9.38491], [51.499, 9.3836099]]","[51.49764495, 9.38425995]"
4,51881672,174713859,174587951,110.17,3,B,6,50,50,2,2,F,T,0.0,53.0643099/8.7903400/45.79|53.0650299/8.791470...,NaN,0.00/-0.090|110.17/0.062,50,"[[53.0643099, 8.79034], [53.0650299, 8.79147]]","[53.0646699, 8.790905]"


In [225]:
# create link data lookup dictionary
links = []
link_db = lc.LinkDatabase()
with open(os.path.join(DATA_DIR, 'Partition6467LinkData.csv'), 'r') as csvfile:
    rdr = csv.DictReader(csvfile, delimiter=',', fieldnames=link_headers)
    for r in rdr:
        rl = lc.RoadLink(r)
        links.append(rl)
        link_db.insert_link(rl)

In [226]:
# create road link graph
node_id_map = {}
with open(os.path.join(DATA_DIR, 'Partition6467LinkData.csv'), 'r') as csvfile:
    rdr = csv.DictReader(csvfile, delimiter=',', fieldnames=link_headers)
    for r in rdr:
        try:
            node_id_map[r['refNodeID']] += [r['linkPVID']]
        except KeyError:
            node_id_map[r['refNodeID']] = [r['linkPVID']]

road_graph = graph.RoadNetwork(node_id_map)
with open(os.path.join(DATA_DIR, 'Partition6467LinkData.csv'), 'r') as csvfile:
    rdr = csv.DictReader(csvfile, delimiter=',', fieldnames=link_headers)
    for r in rdr:
        road_graph.insert(graph.RoadNetworkNode(node_id_map, \
                                                [[float(j) for j in i.split('/')[:2]] for i in r['shapeInfo'].split('|')], \
                                                r['linkPVID'], \
                                                r['nrefNodeID']))

## ST-Matching Algorithm
Below is an implementation of the ST-Matching algorithm from the [Map-Matching for Low_sampling-Rate GPS Trajectories](https://www.microsoft.com/en-us/research/publication/map-matching-for-low-sampling-rate-gps-trajectories/?from=http%3A%2F%2Fresearch.microsoft.com%2Fapps%2Fpubs%2Fdefault.aspx%3Fid%3D105051) paper. 

### Defining useful utility functions

In [227]:
Candidate = namedtuple('Candidate', ['linkPVID', 'location', 'speedLimit'])

def nearest_n_segments(lat, long, n):
    """
    Uses link_db to find nearest n road segments
    
    Inputs:
        lat (float): latitude of probe point
        lon (float): longitude of probe point
        n (int): number of roads road segments to return
        
    Output: 
        (list of Candidate tuples): Candidate tuples of n-nearest road segments 
        
    """
    # find nearest n links
    output = []
    try:
        link_search = [(x, haversine(x.avgLatLong, (lat, long))) for x in link_db.get_links(lat, long)]
        link_search.sort(key=operator.itemgetter(1))
        link_search = link_search[0:n]

        # extract only link PVIDs from search
        output = [(int(x[0].linkPVID), x[1]) for x in link_search]
    except KeyError:
        pass
    
    # format output with additional data
    for i in range(len(output)):
        link_information = link_data[link_data['linkPVID'] == output[i][0]]
        formatted_output = Candidate(int(link_information['linkPVID']), \
                                     list(link_information['location'])[0], \
                                    float(link_information['speedLimit']))
        output[i] = formatted_output
    
    return output

### Defining scoring functions

#### Spatial Analysis

In [228]:
def observation_probability(p, c):
    """
    Compute the likelihood that a GPS point p matches a candidate point c based on the distance between the two.
    
    Inputs:
        p (tuple of lat, long): probe point
        c (tuple of lat, long): candidate point
    
    Output:
        (float): probability of the above
    """
    distance = haversine(p, c)
    return (1 / math.sqrt(2 * math.pi * 20)) * math.exp((distance**2) / (2 * 20**2))

def transmission_probability(p_prev, p_curr, c_prev, c_curr):
    """
    Compute the likelihood that the true path from p_prev to p_curr follows the shortest path from c_prev to c_curr
    
    Inputs:
        p_prev (tuple of lat, long): previous probe point
        p_curr (tuple of lat, long): current probe point
        c_prev (tuple of lat, long): candidate point for previous probe point
        c_curr (tuple of lat, long): candidate point for current probe point
    
    Output:
        (float): probability of the above
    """
    p_dist = haversine(p_prev, p_curr)
    c_dist = haversine(c_prev, c_curr)
    
    if c_dist == 0:
        return 1
    
    return p_dist / c_dist

def spatial_analysis(p_prev, p_curr, c_prev, c_curr):
    """
    Compute the spatial measurement value for two neighboring probe points p_prev and p_curr for
    two candidate points c_prev, c_curr
    
    Inputs:
        p_prev (tuple of lat, long): previous probe point
        p_curr (tuple of lat, long): current probe point
        c_prev (tuple of lat, long): candidate point for previous probe point
        c_curr (tuple of lat, long): candidate point for current probe point
    
    Output:
        (float): spatial measurement value
    """
    return observation_probability(p_curr, c_curr) * transmission_probability(p_prev, p_curr, c_prev, c_curr)

#### Temporal Analysis

In [229]:
def cosine_dist(a, b):
    """
    Compute cosine distance between vectors a, b
    
    Input:
        a (numpy array): vector a
        b (numpy array): vector b
    
    Output:
        (float): cosine distance between a, b
    """
    numerator = np.sum(a * b)
    denominator = math.sqrt(np.sum(a ** 2)) * math.sqrt(np.sum(b ** 2))
    return numerator / denominator

def temporal_analysis(p_prev, p_curr, c_prev, c_curr, speed_prev, speed_curr, delta_t):
    """
    Compute the temporal meaurement value for two neighboring probe points and their candidate points c_prev, c_curr
    
    Inputs:
        c_prev (tuple of lat, long): candidate point for previous probe point
        c_curr (tuple of lat, long): candidate point for current probe point 
        speed_prev (float): speed limit of previous road segment
        speed_curr (float): speed limit of current road segment
        delta_t (float): time between previous probe and current probe point measurments
        
    Output:
        (float): temporal measurment value
    """
    avg_speed = haversine(c_prev, c_curr) / delta_t
    if avg_speed == 0:
        avg_speed = haversine(p_prev, p_curr) / delta_t
    return cosine_dist(np.array([avg_speed, avg_speed]), np.array([speed_prev, speed_curr]))

#### ST Function

In [230]:
def st_function(p_prev, p_curr, c_prev, c_curr, speed_prev, speed_curr, delta_t):
    """
    Computes st measurement
    
    Inputs: 
        p_prev (tuple of lat, long): previous probe point
        p_curr (tuple of lat, long): current probe point
        c_prev (tuple of lat, long): candidate point for previous probe point
        c_curr (tuple of lat, long): candidate point for current probe point 
        speed_prev (float): speed limit of previous road segment
        speed_curr (float): speed limit of current road segment
        delta_t (float): time between previous probe and current probe point measurments
        
    Output:
        (float): st measurement
    """
    return spatial_analysis(p_prev, p_curr, c_prev, c_curr) * \
           temporal_analysis(p_prev, p_curr, c_prev, c_curr, speed_prev, speed_curr, delta_t)

### Main Algorithm

In [231]:
def find_candidates(sample_id, location, delta_t):
    """
    Finds nearest candidates for probe data point
    
    Input: 
        sample_id (string): sample_id
        location (tuple): (lat, long)
        delta_t (float): time diff with last probe
        
    Output: 
        (tuple): (sample_id, (lat, long), delta_t, [Candidates])
    """
    return (sample_id, location, delta_t, nearest_n_segments(location[0], location[1], 3))

def st_matching_algorithm(sample):
    """
    Match trajectory points to road links
    
    Input:
        sample (pandas dataframe): probe data matching a sample id
    
    Output: 
        (list of tuples): list of tuples (sample_id, linkPVID)
    """
    # get list of candidate points
    candidates_for_id = [None for x in range(len(sample))]
    sample_ids = [None for x in range(len(sample))]
    counter = 0
    
    for row in sample.itertuples():
        delta_t = 0
        try:
            delta_t = (sample.loc[row.Index, 'dateTime'] - sample.loc[row.Index - 1, 'dateTime']).seconds
        except KeyError:
            pass
        
        candidates_for_id[counter] = find_candidates(row.id, (row.latitude, row.longitude), delta_t)
        sample_ids[counter] = row.id
        counter += 1

    # find matched sequence    
    matched_sequence = find_matched_sequence(candidates_for_id)
    
    # zip together sample_id and linkPVID
    return zip(sample_ids, matched_sequence)

def find_matched_sequence(candidates):
    """
    Find longest matching sequence given candidates
    
    Input:
        candidates (list of tuple): see find_candidates
    
    Output:
       (list of tuples): list of tuples (sample_id, linkPVID) 
    """
    # setup variables 
    parents = [{str(i.linkPVID): None for i in candidate[3]} for candidate in candidates]
    f = [{str(i.linkPVID): 0 for i in candidate[3]} for candidate in candidates]
    candidate_count = len(candidates)
    
    # set f[0] equal to observation probability
    for c in candidates[0][3]:
        f[0][str(c.linkPVID)] = observation_probability(candidates[0][1], c.location)
    
    # compute scores for each node
    for i in range(1, candidate_count):
        for cs in candidates[i][3]:
            max_val = -math.inf
            for ct in candidates[i - 1][3]:
                # check if cs is a valid neighbor of ct
                if (str(cs.linkPVID) != str(ct.linkPVID)) and (str(cs.linkPVID) not in road_graph.nodes[str(ct.linkPVID)].neighbors):
                    f[i][str(cs.linkPVID)] = max_val
                    continue
                    
                # define all the variables then compute new score
                p_prev = candidates[i - 1][1]
                p_curr = candidates[i][1]
                c_prev = ct.location
                c_curr = cs.location
                speed_prev = ct.speedLimit
                speed_curr = cs.speedLimit
                delta_t = candidates[i][2]
                
                alt = f[i - 1][str(ct.linkPVID)] + st_function(p_prev, p_curr, c_prev, c_curr, speed_prev, speed_curr, delta_t)
                
                # check if higher than existing
                if alt > max_val:
                    max_val = alt
                    parents[i][str(cs.linkPVID)] = ct.linkPVID
                
                # set max value for current node
                f[i][str(cs.linkPVID)] = max_val
        

    # compute path
    r_list = []
    c = max(f[candidate_count - 1], key=f[candidate_count - 1].get)
    for i in range(candidate_count - 1, 0, -1):
        r_list.append(c)
        c = parents[i][str(c)]
    r_list.append(c)
    
    return r_list[::-1]

In [235]:
temp = st_matching_algorithm(probe_data[probe_data['sampleID'] == 4552])
stacked_values = np.dstack(temp)[0]

try:
    del probe_data['linkPVID']
except KeyError:
    pass

probe_data = probe_data.merge(pd.DataFrame({'id': stacked_values[0], 'linkPVID': stacked_values[1]}), how='left', on=['id'])
probe_data['linkPVID'].fillna(0, inplace=True)
probe_data['linkPVID'] = probe_data['linkPVID'].astype(int)

KeyError: 'None'

### Putting it all together

In [40]:
# time code
t0 = time.time()

# sample only first sample_size to make computation faster
sample_size = 1000
# sample_size = len(probe_data) # for all data

# add road link
probe_data['linkPVID'] = 0

# parallelizable function
def link_road_parallel(indicies):
    """
    Links road to probe for set of indicies
    
    Input:
        indicies (list of floats): indicies to find nearest link for
    """
    output = [(0, 0) for x in range(indicies[1] - indicies[0])]
    n = 3
    counter = 0
    for row in probe_data[indicies[0]:indicies[1]].itertuples():
        output[counter] = (row.Index, closest_by_heading(nearest_n_segments(row.latitude, row.longitude, n),
                                                         row.heading))
        counter += 1
    
    return output

# run in parallel
N_CORES = mp.cpu_count()
C_SIZE = math.ceil(sample_size / N_CORES)

pool = mp.Pool(N_CORES)
r = pool.map(link_road_parallel, [[(C_SIZE * i), ((i + 1) * C_SIZE)] for i in range(N_CORES)])
linkings = list(itertools.chain.from_iterable(r))

# assign values to probe_data
stacked_values = np.dstack(linkings)[0]
probe_data.loc[stacked_values[0], 'linkPVID'] = stacked_values[1]
        
# finish timing
t1 = time.time()
print(str((t1 - t0) / 60) + ' minutes for ' + str(sample_size) + ' data points using ' + str(N_CORES) + ' CPU threads.')

0.2601946830749512 minutes for 1000 data points using 8 CPU threads.


In [14]:
probe_data[0:sample_size].to_csv('./sample_linked_data.csv', index=False)

### Create output file
The output file has the following columns (columns in **bold** are pulled from the LinkData csv):
- sampleID: a unique identifier for the set of probe points that were collected from a particular phone.
- dateTime: date and time that the probe point was collected.
- sourceCode: a unique identifier for the data supplier (13 = Nokia).
- latitude: latitude in decimal degrees.
- longitude: longitude in decimal degrees.
- altitude: altitude in meters.
- speed: speed in KPH.
- heading: heading in degrees.
- linkPVID: published versioned identifier for the link.
- **direction**: direction the vehicle was travelling on thelink (F = from ref node, T = towards ref node).
- **distFromRef**: distance from the reference node to the map-matched probe point location on the link in decimal meters.
- **distFromLink**: perpendicular distance from the map-matched probe point location on the link to the probe point in decimal meters.

### Plot data
Now, we plot both the probe data with its associated links

In [233]:
def make_map_plot(method, sample_id, gmaps_api_key, data):
    probe_plot_data = data[(data['linkPVID'] != 0) & (data['sampleID'] == sample_id)]

    # create map object centered at mean lat, long
    gmap = gmplot.GoogleMapPlotter(np.mean(probe_plot_data['latitude']), np.mean(probe_plot_data['longitude']), 16)

    # plot data with color-coded probes and links
    unique_links = probe_plot_data['linkPVID'].unique()
    colors = list(gmap.color_dict.keys())[0:-1]
    color_index = 0

    for i in unique_links:
        # setup variables
        current_color = colors[color_index]
        probe_lats = probe_plot_data[probe_plot_data['linkPVID'] == i]['latitude']
        probe_longs = probe_plot_data[probe_plot_data['linkPVID'] == i]['longitude']

        link_lats = [x[0] for x in list(link_data[link_data['linkPVID'] == i]['shapeArray'])[0]]
        link_longs = [x[1] for x in list(link_data[link_data['linkPVID'] == i]['shapeArray'])[0]]
        
        gmap.scatter(probe_lats, probe_longs, marker=False, color=current_color, s=5)
        gmap.plot(link_lats, link_longs, color=current_color, edge_width=10, alpha=0.25)

        color_index = (color_index + 1) % len(colors)
        print('Link Segment: ' + str(i) + ', Color: ' + str(current_color))
    
    # print out file
    if not os.path.exists('./graphs'):
        os.makedirs('./graphs')
    file_name = './graphs/' + method + '_' + str(sample_id) + '.html'
    gmap.draw(file_name)

    def insertapikey(fname, apikey):
        """put the google api key in a html file"""
        def putkey(htmltxt, apikey, apistring=None):
            """put the apikey in the htmltxt and return soup"""
            if not apistring:
                apistring = 'https://maps.googleapis.com/maps/api/js?key=%s&callback=initMap'
            soup = BeautifulSoup(htmltxt, 'html.parser')
            body = soup.body
            src = apistring % (apikey, )
            tscript = soup.new_tag('script', src=src, async='defer')
            body.insert(-1, tscript)
            return soup
        htmltxt = open(fname, 'r').read()
        soup = putkey(htmltxt, apikey)
        newtxt = soup.prettify()
        open(fname, 'w').write(newtxt)

    insertapikey(file_name, gmaps_api_key)
    return IFrame(file_name, width=985, height=700)

In [234]:
# select data to plot
sample_id = 3496
make_map_plot('st-matching', sample_id, GOOGLE_MAPS_KEY, probe_data)

Link Segment: 62007648, Color: b
Link Segment: 62007637, Color: g
